In [1]:
import os
import argparse
import gym
import numpy as np
from itertools import count
from collections import namedtuple

from TD3_keras.td3 import TD3

args = {
    'render': True,
    'log_interval': 1
}

env = gym.make('BipedalWalker-v2')
episodes = 100000
reward_history = []

task = {
        'state_size': 24,
        'action_size': 4,
        'action_high': 1,
        'action_low': -1
    }
agent = TD3(task)    


def main(agent):   
    
    
    for i_episode in range(episodes):
        running_reward = 0        
        state = env.reset()
        for t in range(10000):  # Don't infinite loop while learning
            action, noise_coeff = agent.act(state, i_episode)                
            state, reward, done, _ = env.step(action)  
            agent.step(action, reward, state, done, t)
            if args['render']:
                env.render()                   
            running_reward += reward            
            if done:
                break
        
        reward_history.append(running_reward)
        
        if i_episode % args['log_interval'] == 0:
            avg_reward = np.mean(reward_history[-100:])            
            print('Episode {}\tLast length: {:5d}\t Reward: {:7.2f}\t Avg Reward: {:7.2f}\t Noise: {:.2f}'.format(
                i_episode, t, running_reward, avg_reward, noise_coeff))
        if avg_reward > env.spec.reward_threshold and i_episode > 100:
            print("Solved! Average 100-episode reward is now {}!".format(avg_reward))
            break
            
print("action_space={}".format(env.action_space))
print("obs_space={}".format(env.observation_space))
print("threshold={} \n".format(env.spec.reward_threshold))
main(agent)



Using TensorFlow backend.


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
action_space=Box(4,)
obs_space=Box(24,)
threshold=300 

Episode 0	Last length:   686	 Reward: -157.88	 Avg Reward: -157.88	 Noise: 1.00
Episode 1	Last length:   119	 Reward: -110.27	 Avg Reward: -134.08	 Noise: 1.00
Episode 2	Last length:    78	 Reward: -115.11	 Avg Reward: -127.75	 Noise: 1.00
Episode 3	Last length:    46	 Reward: -113.45	 Avg Reward: -124.18	 Noise: 1.00
Episode 4	Last length:    52	 Reward: -111.54	 Avg Reward: -121.65	 Noise: 1.00
Episode 5	Last length:    39	 Reward: -108.78	 Avg Reward: -119.51	 Noise: 0.99
Episode 6	Last length:    99	 Reward: -117.96	 Avg Reward: -119.28	 Noise: 0.99
Episode 7	Last length:    48	 Reward: -109.73	 Avg Reward: -118.09	 Noise: 0.99
Episode 8	Last length:   202	 Reward: -116.69	 Avg Reward: -117.94	 Noise: 0.99
Episode 9	Last length:    

Episode 98	Last length:   129	 Reward: -129.83	 Avg Reward: -128.56	 Noise: 0.90
Episode 99	Last length:  1599	 Reward: -150.79	 Avg Reward: -128.78	 Noise: 0.90
Episode 100	Last length:  1599	 Reward: -145.95	 Avg Reward: -128.66	 Noise: 0.90
Episode 101	Last length:  1599	 Reward: -148.81	 Avg Reward: -129.05	 Noise: 0.90
Episode 102	Last length:  1599	 Reward: -149.37	 Avg Reward: -129.39	 Noise: 0.90
Episode 103	Last length:    41	 Reward: -112.33	 Avg Reward: -129.38	 Noise: 0.90
Episode 104	Last length:  1599	 Reward: -150.69	 Avg Reward: -129.77	 Noise: 0.90
Episode 105	Last length:  1599	 Reward: -154.84	 Avg Reward: -130.23	 Noise: 0.90
Episode 106	Last length:  1599	 Reward: -142.08	 Avg Reward: -130.47	 Noise: 0.89
Episode 107	Last length:  1599	 Reward: -150.30	 Avg Reward: -130.88	 Noise: 0.89
Episode 108	Last length:  1599	 Reward: -151.19	 Avg Reward: -131.22	 Noise: 0.89
Episode 109	Last length:  1599	 Reward: -149.02	 Avg Reward: -131.63	 Noise: 0.89
Episode 110	Last l

KeyboardInterrupt: 

In [ ]:
from PIL import Image
def test(agent):   
    random_seed = 0
    episodes = 3
    max_timesteps = 2000
    render = True
    save_gif = True
     
    for i_episode in range(1, episodes):
        running_reward = 0        
        state = env.reset()
        for t in range(10000):  # Don't infinite loop while learning
            action, noise_coeff = agent.act(state, i_episode)                
            state, reward, done, _ = env.step(action)  
            agent.step(action, reward, state, done, t)
            if args['render']:
                env.render()  
                if save_gif:
                    dirname = './gif/td3_keras/{}'.format(i_episode)
                    if not os.path.isdir(dirname):
                        os.mkdir(dirname)
                    img = env.render(mode = 'rgb_array')
                    img = Image.fromarray(img)
                    img.save('./gif/td3_keras/{}/{}.jpg'.format(i_episode,t))
            running_reward += reward            
            if done:
                break    
   
            
        print('Episode: {}\tReward: {}'.format(i_episode, int(running_reward)))
        running_reward = 0
        env.close()        
                
test(agent)